In [2]:
%pip install datasets transformers torch evaluate nltk rouge_score scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
import numpy as np
import evaluate
from ctransformers import AutoModelForCausalLM

# Setup evaluation
metric = evaluate.load("accuracy")

# checkpoint = "microsoft/codebert-base"
# checkpoint = "bert-base-uncased"
checkpoint = "TheBloke/CodeLlama-7B-GGUF"
dataset_name = "microsoft/LCC_csharp"

# Prepare and tokenize dataset
dataset = load_dataset(dataset_name)
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")


def tokenize_function(examples):
    return tokenizer(examples["context"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = (
    tokenized_datasets["train"].shuffle(seed=42, keep_in_memory=True).select(range(200))
)
small_eval_dataset = (
    tokenized_datasets["test"].shuffle(seed=42, keep_in_memory=True).select(range(200))
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


# Load pretrained model and evaluate model after each epoch
# model = AutoModelForTokenClassification.from_pretrained(checkpoint, num_labels=1)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    model_file="codellama-7b.q4_K_M.gguf",
    model_type="llama",
    gpu_layers=0,
    hf=True,
)
training_args = TrainingArguments(
    output_dir="test_trainer", evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

c:\Users\shafi\source\Notebooks\.venv\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


OSError: [Errno 28] No space left on device